# Analysis of Mobile Apps

**Functional Goal**: We are interested in apps with a maximal number of users. We assume that all apps are free to download, and generate income based on the size of its userbase.

In this analysis, we use datasets from the App Store and Google Play to better understand which genres have a larger number of users.

**Datasets**: 
* A sampling of 10,000 Android apps from Google Play; collected on August 2018. [Link to download .csv file](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* A sampling of 7,000 iOS apps from the App Store; collected on July 2017. [Link to download .csv file](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

The entirety of this analysis was done **sans Pandas**. Instead, the datasets were analyzed using pure Python and bespoke functions, with a touch of help from very common packages. Let's begin by reading in the datasets and importing some basic libraries.

In [1]:
import csv
import os
import requests

Define a function to repeatedly load in datasets, and convert them to a list-of-lists for easier, iterative analysis.

In [2]:
def open_data(dataset, headers=True):
    '''
    A function to convert .csv files to a list-of-lists.

    Args:
        dataset (str): Relative path to a dataset in .csv format.
        headers (bool): Function returns first row of dataset, by default. Set to "False" to return only row with index 1 and higher.
        
    Dependencies:
        import csv

    Returns:
        multi_list (list): A list of lists.
    '''
    opened_file = open(dataset, encoding='utf8')
    read_file = csv.reader(opened_file)
    apps_data = list(read_file)

    if headers:
        return apps_data
    else:
        return apps_data[1:]


Load in the data and save the list-of-lists to separate variables.

In [3]:
android = open_data('./../googleplaystore.csv')
apple = open_data('./../AppleStore.csv')

Define a function for creating a data overview.

In [4]:
def explore_data(dataset, start, end, rows_and_columns=False):
    '''
    A function to look at a subset of rows in a list of lists.

    Args:
        dataset (list): A list of lists containing data. 
        start (int): The first row to include in a slice of the data view. 
        end (int): The last row to include in a slice of the data view. 
        rows_and_columns (bool): Return the number of rows and columns in a dataset? (y/n)

    Returns:
        None
    '''
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print(f"Number of rows: {len(dataset)}")
        print(f"Number of columns: {len(dataset[0])}")

In [5]:
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


In [6]:
explore_data(apple, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16


### Detecting Duplicates

Goals: 
* Remove non-English apps.
* Remove paid apps.

One of the rows in the dataset may have an incorrect value, leading to a frameshift in the `ratings` column. See this [Kaggle post](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) for more details.

In [7]:
explore_data(android, 10470, 10475)

['TownWiFi | Wi-Fi Everywhere', 'COMMUNICATION', '3.9', '2372', '58M', '500,000+', 'Free', '0', 'Everyone', 'Communication', 'August 2, 2018', '4.2.1', '4.2 and up']


['Jazz Wi-Fi', 'COMMUNICATION', '3.4', '49', '4.0M', '10,000+', 'Free', '0', 'Everyone', 'Communication', 'February 10, 2017', '0.1', '2.3 and up']


['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']




The index of the row with the error in the `android` dataset is 10473.

In [8]:
if 'Life Made WI-Fi Touchscreen Photo Frame' in android[10473]:
    del android[10473]
    print(len(android))

10841


Some apps are also duplicated; take Instagram as an example.

In [9]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


By looking at each of the duplicated Instagram entries, we notice something important: The column with index = 3 is the only bit that changes. This is the `ratings` column, and we can assume that the highest value here corresponds to the most recent data. In a future step, we will remove all duplicate entries, and keep only that row that has the highest number of reviews.

Instead of trying to find each app that is duplicated, though, we can write a function that iterates through the list, checks whether each app has already been viewed, and marks duplicates.

In [12]:
duplicate_apps = []
unique_apps = []

# Iterate through the dataset, but remove the header row. 
for app in android[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

In [13]:
print(f"There are {len(duplicate_apps)} duplicated apps.")
print(f"There are {len(unique_apps)} unique apps in the Android dataset.")
print(f"Examples of duplicate apps: {duplicate_apps[:10]}")

There are 1181 duplicated apps.
There are 9659 unique apps in the Android dataset.
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


### Removing Duplicates

Define a function that detects whether an app is already in the dataset. Remove it unless it has the highest number of ratings out of all the duplicate rows.

In [22]:
def find_unique_apps(dataset, name_column, value_column, print_results=False, header=True):
    '''
    A function that identifies unique values in a list-of-lists, and adds them to a dictionary.

    Args:
        dataset (list): A list of lists containing data. 
        name_column (int): The column name in which duplicates may appear (e.g. the list of app names in our example.)
        value_column (int): The column to store as values, int, in the comparator dictionary (e.g. the number of ratings for each app in our example.)
        print_results (bool): Print sentences displaying the before-and-after results? (y/n)
        header (bool): If header is present, remove first row of dataset by default. If header is not present, continue.

    Returns:
        dataset (dict): A deduplicated dictionary with key:value pairs. The value is the highest value for the items with a given key in the initial dataset.
    '''
    reviews_max = {}

    if header:
        dataset = dataset[1:]

    for row in dataset:
        name = row[name_column]
        n_reviews = float(row[value_column])

        if name not in reviews_max:
            reviews_max[name] = n_reviews
        elif name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews

    if print_results:
        print(f"Length of initial dataset: {len(dataset)}")
        print(f"Length of dataset with only unique apps: {len(reviews_max)}")

    return reviews_max

In [23]:
apple_uniques = find_unique_apps(apple, 0, 5, print_results=True)
android_uniques = find_unique_apps(android, 0, 3, print_results=True)

Length of initial dataset: 7197
Length of dataset with only unique apps: 7197
Length of initial dataset: 10840
Length of dataset with only unique apps: 9659


Our function has created a dictionary that contains key:value pairs corresponding to the name:no_of_ratings for each app in the Android and Apple datasets. Each key:value pair was replaced if, and only if, the same name appeared again with a higher number of ratings. Thus, we've retained those entries in which no_of_ratings was highest, as we are assuming that these correspond to the most recent app in the dataset.

Note that the apple dataset did not have any duplicate values. Now that's clean!

Now, I use these dictionaries to remove duplicate applications.

In [27]:
"""
STRATEGY

Create an empty list. This will store our new list-of-lists with deduplicated values.
Iterate through each row in the original list-of-lists. 
Store the name and no. of reviews for each row into a variable within the for loop. 
Check if the name is in the dictionary. If it is, check if the no. of reviews matches. 
If reviews match: Append the entire row, from the initial dataset, to the clean list. 
If reviews do not match: Continue 
Return the clean dataset. 
"""

def deduplicate_data(dataset, unique_dict, index_1, index_2, print_results=False, header=True):
    """
    A function that removes duplicate values from a list of lists, keeping only that row that has the highest value from a specified column.

    Args:
        dataset (list): A list of lists containing the original data with duplicated values. 
        unique_dict (dict): A dictionary containing the key:value pairs with unique values.
        index_1 (int): The column used as keys in the unique_dict.
        index_2 (int): The column used as values in the unique_dict.
        print_results (bool): Print sentences displaying the before-and-after results.
        header (bool): If header is present, remove first row of dataset by default. If header is not present, continue.

    Returns:
        clean_list (list): A list containing deduplicated rows.
    """
    clean_list = []
    already_added = []

    if header:
        dataset = dataset[1:]

    for row in dataset:
        name = row[index_1]
        no_of_ratings = float(row[index_2])

        if name in unique_dict and no_of_ratings == unique_dict[name] and name not in already_added:
            clean_list.append(row)
            already_added.append(name)

    print(f"Length of unique list: {len(already_added)}")

    if print_results:
        print(f"The length of clean_list is: {len(clean_list)}")

    return clean_list
    

In [29]:
android_clean = deduplicate_data(android, android_uniques, 0, 3, print_results=True)

Length of unique list: 9659
The length of clean_list is: 9659


In [52]:
# The headers are not present in this dataset anymore. 
print(android_clean[0])
print(apple[0])

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [53]:
apple = apple[1:]

### Remove non-English apps

For this phase, we'll use [ASCII](https://en.wikipedia.org/wiki/ASCII) criteria. English characters, like A->Z and 0->9, are all in the range of 0 to 127. If you execute the built-in `ord()` function on a character, you can see its ASCII value. Example:

In [26]:
print(ord('a'))
print(ord('Z'))
print(ord('5'))

97
90
53


Let's define a threshold. We shall iterate through our datasets and remove those apps that have more than **three** non-English characters in the name.

In [49]:
def is_english(string, threshold=3):
    """
    A function that checks whether a given string contains a non-English character. Tunable threshold.

    Args:
        string (str): The string to be checked.
        threshold (int): The permittable number of non-English characters before an app is removed from the list.

    Returns:
        english_bool (bool): True or False, does the list contain all English characters. True if yes.
    """
    counter = 0
    for character in string:
        if ord(character) > 127:
            counter += 1

    if counter > threshold:
        return False
    else:
        return True

Test the function on basic cases.

In [47]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


Now we iterate through each dataset and remove each non-English app using the custom function, `is_english`

In [54]:
def filter_english_apps(dataset, index_col, header=False):
    """
    A function that iterates through a dataset and removes rows with non-English index_col values.

    Args:
        dataset (list): The dataset to be checked.
        index_col (int): The column to be checked for non-English characters
        header (bool): If header is present, remove first row of dataset by default. If header is not present, continue.

    Returns:
        filtered_data (list): A list containing only apps with English names.
    """
    if header:
        dataset = dataset[1:]

    filtered_data = []

    for row in dataset:
        name = row[index_col]

        if is_english(name):
            filtered_data.append(row)

    return filtered_data

In [55]:
android_english = filter_english_apps(android_clean, 0)
apple_english = filter_english_apps(apple, 0)

In [56]:
print(f"Length of android list with only English apps: {len(android_english)}")
print(f"Length of android list with only English apps: {len(apple_english)}")

Length of android list with only English apps: 9614
Length of android list with only English apps: 7197


### Remove Paid Apps

In the Android dataset, the `Price` column gives apps either as 'Free' (str) or as $3.99, for example. So I will need to remove the dollar symbol and convert to floats.

In the Apple dataset, the `price` column gives apps as either 0.0 (float), which denotes free, or as 0.99, for example. There are no dollar signs.

I will need to devise a function that is flexible enough to handle both cases.

In [61]:
def remove_paid_apps(dataset, index_col, header=False, print_results=True):
    """
    A function that iterates through a dataset and removes paid apps.

    Args:
        dataset (list): The dataset to be checked.
        index_col (int): The column to be checked for prices.
        header (bool): If header is present, remove first row of dataset by default. If header is not present, continue.
        print_results (bool): Print before and after results.

    Returns:
        clean_data (list): A list containing only FREE apps.
    """
    if header:
        dataset = dataset[1:]

    clean_data = []

    for row in dataset:
        check_val = row[index_col]
        if '$' in check_val:
            check_val = check_val.replace("$", "")

        if check_val == 'Free' or float(check_val) == 0.0:
            clean_data.append(row)

    print(f"Number of rows before: {len(dataset)}")
    print(f"Number of rows after: {len(clean_data)}")
    
    return clean_data

In [62]:
android_free_english = remove_paid_apps(android_english, 7)
apple_free_english = remove_paid_apps(apple_english, 4)

Number of rows before: 9614
Number of rows after: 8864
Number of rows before: 7197
Number of rows after: 4056


### Data Exploration

To minimize risks and overhead, we want to find app profiles that do well on both Android and Apple marketplaces. When we develop our app, there's three things we'd like to do:

* Build a minimal Android version of the app, and add it to Google Play.
* If the app has a good response from users, we develop it further.
* If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Each dataset has specific columns that will be important to analyze. We'd like to know the `genre` for each app, for instance, so we can determine whether some categories are more popular than others in terms of downloads. We can also base our genre analysis on the number of reviews giving to the app, or only look at apps that are 4 stars+ with a set threshold number of reviews.

We need to create two functions: One that generates frequency tables and spits out percentages. And another function that displays those returned percentages in descending order.

In [72]:
# Create a function named freq_table() that takes two inputs: dataset and index
# The function should return a dictionary with the keys being the values in the index column and the values being the frequency of the key, as a percentage.
def freq_table(dataset, index):
    """
    A function that creates a frequency table of the values in a given column.

    Args:
        dataset (list): The dataset to be checked.
        index (int): The column to be checked for values.

    Returns:
        dict_pct (dict): A dictionary containing each key and its value as a percentage.
    """
    freq_dict = {}
    dict_pct = {}

    for row in dataset:
        value = row[index]

        if value in freq_dict:
            freq_dict[value] += 1
        else:
            freq_dict[value] = 1

    # Calculate the percentage of each key in freq_dict and add to dict_pct
    for key in freq_dict:
        dict_pct[key] = round((freq_dict[key] / sum(freq_dict.values())) * 100, 2)

    return dict_pct

In [73]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Printing and Analyzing Results

Begin by analyzing the `prime_genre` column for the Apple Store.

In [77]:
display_table(apple_free_english, 11)

Games : 55.65
Entertainment : 8.23
Photo & Video : 4.12
Social Networking : 3.53
Education : 3.25
Shopping : 2.98
Utilities : 2.69
Lifestyle : 2.32
Finance : 2.07
Sports : 1.95
Health & Fitness : 1.87
Music : 1.65
Book : 1.63
Productivity : 1.53
News : 1.43
Travel : 1.38
Food & Drink : 1.06
Weather : 0.76
Reference : 0.49
Navigation : 0.49
Business : 0.49
Catalogs : 0.22
Medical : 0.2


**Analysis**: Games are the most popular category, by far. This is followed by Enterntainment. At least two-thirds of all apps are designed for Entertainment, rather than practical purposes.

In [74]:
display_table(android_free_english, 9)

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;

In [76]:
display_table(android_free_english, 1)

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


Games : 55.65
Entertainment : 8.23
Photo & Video : 4.12
Social Networking : 3.53
Education : 3.25
Shopping : 2.98
Utilities : 2.69
Lifestyle : 2.32
Finance : 2.07
Sports : 1.95
Health & Fitness : 1.87
Music : 1.65
Book : 1.63
Productivity : 1.53
News : 1.43
Travel : 1.38
Food & Drink : 1.06
Weather : 0.76
Reference : 0.49
Navigation : 0.49
Business : 0.49
Catalogs : 0.22
Medical : 0.2
